In [48]:
import time
import nltk
from nltk.corpus import stopwords

#NLP
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.collocations import *
import string, re

from sklearn.feature_extraction.text import TfidfVectorizer

#Model Data Prep
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

%matplotlib inline
np.random.seed(0)

import re

from sklearn.metrics import accuracy_score, classification_report

In [4]:
jobs = pd.read_csv("jobs.csv")
jobs

,JOB_NAME,COMPANY,LOCATION,DESCRIPTION,SALARY
0,Chief Financial Officer,Community Memorial Hospital,"Hamilton, NY 13346",Provide visionary leadership to the finance te...,"$150,000 - $200,000 a year"
1,SENIOR CONSULTANT- VALUATION AND FORENSIC SERV...,Guidepost Solutions LLC,"Hybrid remote in New York, NY 10016",Five (5) to ten (10) years’ of industry/sector...,"$115,000 - $120,000 a year"
2,Universal Banker,ESL Federal Credit Union,"Rochester, NY 14604 (Central Business District...",Monday - Friday to follow branch hours of Mond...,$21.17 an hour
3,Controller,Shine Electronics Co.,"Hybrid remote in Long Island City, NY 11101",S/he will work cross functionally with Operati...,"$130,000 - $150,000 a year"
4,Financial Analyst,"Neotecra, Inc","Remote in Bethpage, NY",Financial practices: 5 years (Preferred).\nThe...,Salary not available
...,...,...,...,...,...
37470,AI & Digital Tools Intern,Capria Ventures,"Seattle, WA",Attention to detail: Researching digital tools...,Salary not available
37471,Intern - CPU Profiler Software Engineer (NTD),Nintendo,"Redmond, WA","Finally, candidates need strong problem solvin...",$40 an hour
37472,Strategic Communications Intern - Summer 2024,HNTB Corporation,"Bellevue, WA 98005 (Wilburton area)",Participate in team meetings and strategy disc...,$17.67 - $30.47 an hour
37473,Gene Delivery Process and Analytical Developme...,Bristol Myers Squibb,"Seattle, WA 98102 (South Lake Union area)",Gene Delivery Process and Analytical Developme...,$20 - $29 an hour


In [33]:
jobs_filtered = jobs[jobs['SALARY'] != 'Salary not available']
jobs_filtered.drop_duplicates(subset=['JOB_NAME', 'COMPANY', 'LOCATION'], inplace=True)
jobs_filtered

/Users/kevinqueally/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,JOB_NAME,COMPANY,LOCATION,DESCRIPTION,SALARY
0,Chief Financial Officer,Community Memorial Hospital,"Hamilton, NY 13346",Provide visionary leadership to the finance te...,"$150,000 - $200,000 a year"
1,SENIOR CONSULTANT- VALUATION AND FORENSIC SERV...,Guidepost Solutions LLC,"Hybrid remote in New York, NY 10016",Five (5) to ten (10) years’ of industry/sector...,"$115,000 - $120,000 a year"
2,Universal Banker,ESL Federal Credit Union,"Rochester, NY 14604 (Central Business District...",Monday - Friday to follow branch hours of Mond...,$21.17 an hour
3,Controller,Shine Electronics Co.,"Hybrid remote in Long Island City, NY 11101",S/he will work cross functionally with Operati...,"$130,000 - $150,000 a year"
5,Senior Manager- Financial Services Tax Technol...,EisnerAmper LLP,"New York, NY 10017 (Turtle Bay area)",Provide regular reports on status of testing t...,"$131,731 a year"
...,...,...,...,...,...
37468,Business Strategy Intern,Terex Corporation,"Bothell, WA",The compensation for this position is $26/hr f...,$26 an hour
37471,Intern - CPU Profiler Software Engineer (NTD),Nintendo,"Redmond, WA","Finally, candidates need strong problem solvin...",$40 an hour
37472,Strategic Communications Intern - Summer 2024,HNTB Corporation,"Bellevue, WA 98005 (Wilburton area)",Participate in team meetings and strategy disc...,$17.67 - $30.47 an hour
37473,Gene Delivery Process and Analytical Developme...,Bristol Myers Squibb,"Seattle, WA 98102 (South Lake Union area)",Gene Delivery Process and Analytical Developme...,$20 - $29 an hour


In [46]:
def toYearlySalary(rawSalary):
    # Pattern to find decimal/integer numbers allowing commas and ignoring negative signs
    pattern = r'-?\b\d{1,3}(?:,\d{3})*(?:\.\d+)?\b' 
    # Find all pattern matches in the salary text
    matches = re.findall(pattern, rawSalary)
    # Extract them and convert to positive numbers (ignoring - sign)
    nums = [float(match.replace(',', '')) for match in matches]
    # Find average of the numbers
    avg = sum(nums) / len(nums) if nums else 0
    
    # Defining a multiplier for salary conversions
    multiplierDict = {
        "year" : 1,
        "yearly" : 1,
        "month" : 12,
        "monthly" : 12,
        "week" : 52,
        "weekly" : 52,
        "day" : 260,
        "daily" : 260,
        "hour" : 2080,
        "hourly" : 2080
    }
    # Finding multiplier
    multiplier = 0
    for word in rawSalary.split():
        if word.lower() in multiplierDict:
            multiplier = multiplierDict[word.lower()]
            break  # Exit loop after finding the multiplier
            
    return avg * multiplier

def toSalaryBin(yearlySalary):
    if yearlySalary < 40000:
        return "Under $40,000"
    elif yearlySalary < 60000:
        return "$40,000 - $60,000"
    elif yearlySalary < 80000:
        return "$60,000 - $80,000"
    elif yearlySalary < 100000:
        return "$80,000 - $100,000"
    elif yearlySalary < 120000:
        return "$100,000 - $120,000"
    elif yearlySalary < 140000:
        return "$120,000 - $140,000"
    elif yearlySalary < 180000:
        return "$140,000 - $180,000"
    else:
        return "Over $180,000"
    
    
    
jobs_filtered['YEARLY_SALARY'] = jobs_filtered['SALARY'].apply(lambda x : toYearlySalary(x))
jobs_filtered['SALARY_BIN'] = jobs_filtered['YEARLY_SALARY'].apply(lambda x : toSalaryBin(x))
jobs_filtered = jobs_filtered[jobs_filtered['YEARLY_SALARY'] > 0]
jobs_filtered['JCL'] = jobs_filtered['JOB_NAME'] + ' ' + jobs_filtered['COMPANY'] + ' ' + jobs_filtered['LOCATION']
jobs_filtered['All'] = jobs_filtered['JCL'] + ' ' + jobs_filtered['DESCRIPTION']
jobs_final = jobs_filtered
jobs_final

,JOB_NAME,COMPANY,LOCATION,DESCRIPTION,SALARY,YEARLY_SALARY,SALARY_BIN,JCL,All
0,Chief Financial Officer,Community Memorial Hospital,"Hamilton, NY 13346",Provide visionary leadership to the finance te...,"$150,000 - $200,000 a year",175000.0,"$140,000 - $180,000",Chief Financial Officer Community Memorial Hos...,Chief Financial Officer Community Memorial Hos...
1,SENIOR CONSULTANT- VALUATION AND FORENSIC SERV...,Guidepost Solutions LLC,"Hybrid remote in New York, NY 10016",Five (5) to ten (10) years’ of industry/sector...,"$115,000 - $120,000 a year",117500.0,"$100,000 - $120,000",SENIOR CONSULTANT- VALUATION AND FORENSIC SERV...,SENIOR CONSULTANT- VALUATION AND FORENSIC SERV...
2,Universal Banker,ESL Federal Credit Union,"Rochester, NY 14604 (Central Business District...",Monday - Friday to follow branch hours of Mond...,$21.17 an hour,44033.6,"$40,000 - $60,000",Universal Banker ESL Federal Credit Union Roch...,Universal Banker ESL Federal Credit Union Roch...
3,Controller,Shine Electronics Co.,"Hybrid remote in Long Island City, NY 11101",S/he will work cross functionally with Operati...,"$130,000 - $150,000 a year",140000.0,"$140,000 - $180,000",Controller Shine Electronics Co. Hybrid remote...,Controller Shine Electronics Co. Hybrid remote...
5,Senior Manager- Financial Services Tax Technol...,EisnerAmper LLP,"New York, NY 10017 (Turtle Bay area)",Provide regular reports on status of testing t...,"$131,731 a year",131731.0,"$120,000 - $140,000",Senior Manager- Financial Services Tax Technol...,Senior Manager- Financial Services Tax Technol...
...,...,...,...,...,...,...,...,...,...
37468,Business Strategy Intern,Terex Corporation,"Bothell, WA",The compensation for this position is $26/hr f...,$26 an hour,54080.0,"$40,000 - $60,000",Business Strategy Intern Terex Corporation Bot...,Business Strategy Intern Terex Corporation Bot...
37471,Intern - CPU Profiler Software Engineer (NTD),Nintendo,"Redmond, WA","Finally, candidates need strong problem solvin...",$40 an hour,83200.0,"$80,000 - $100,000",Intern - CPU Profiler Software Engineer (NTD) ...,Intern - CPU Profiler Software Engineer (NTD) ...
37472,Strategic Communications Intern - Summer 2024,HNTB Corporation,"Bellevue, WA 98005 (Wilburton area)",Participate in team meetings and strategy disc...,$17.67 - $30.47 an hour,50065.6,"$40,000 - $60,000",Strategic Communications Intern - Summer 2024 ...,Strategic Communications Intern - Summer 2024 ...
37473,Gene Delivery Process and Analytical Developme...,Bristol Myers Squibb,"Seattle, WA 98102 (South Lake Union area)",Gene Delivery Process and Analytical Developme...,$20 - $29 an hour,50960.0,"$40,000 - $60,000",Gene Delivery Process and Analytical Developme...,Gene Delivery Process and Analytical Developme...


In [50]:
#Categorical target
target_class = jobs_final['SALARY_BIN'].astype('category')
text = jobs_final['JCL']

X_train, X_test, y_train, y_test = train_test_split(text, target_class, test_size=0.1, random_state=4)
tdidf = TfidfVectorizer(sublinear_tf = True, min_df=0.01, max_df=0.5, ngram_range=(1,3), stop_words='english')
fitted_vectorizer = tdidf.fit(X_train)
tfidf_vectorizer_vectors = fitted_vectorizer.transform(X_train)

model = SGDClassifier(alpha=0.0001, max_iter=500, n_jobs=3).fit(tfidf_vectorizer_vectors, y_train)


# Transform the test set using the fitted vectorizer
X_test_transformed = fitted_vectorizer.transform(X_test)

# Make predictions
predictions = model.predict(X_test_transformed)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

# Generate a classification report
print("Classification Report:")
print(classification_report(y_test, predictions))


Accuracy: 0.2797586396050466
Classification Report:
                     precision    recall  f1-score   support

$100,000 - $120,000       0.19      0.21      0.20       234
$120,000 - $140,000       0.10      0.09      0.09       162
$140,000 - $180,000       0.21      0.18      0.19       208
  $40,000 - $60,000       0.39      0.63      0.48       355
  $60,000 - $80,000       0.20      0.14      0.16       342
 $80,000 - $100,000       0.21      0.19      0.20       280
      Over $180,000       0.44      0.50      0.47       157
      Under $40,000       0.38      0.12      0.18        85

           accuracy                           0.28      1823
          macro avg       0.27      0.26      0.25      1823
       weighted avg       0.26      0.28      0.26      1823



In [64]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error

# Features and target
text = jobs_final['JCL']
target_salary = jobs_final['YEARLY_SALARY']

# Splitting data
X_train, X_test, y_train, y_test = train_test_split(text, target_salary, test_size=0.1, random_state=4)

# TF-IDF Vectorization
tdidf = TfidfVectorizer(sublinear_tf=True, min_df=0.01, max_df=0.5, ngram_range=(1, 4), stop_words='english')
fitted_vectorizer = tdidf.fit(X_train)
tfidf_vectorizer_vectors = fitted_vectorizer.transform(X_train)

# Model - using SGDRegressor for regression
model = SGDRegressor(alpha=0.0001, max_iter=1000).fit(tfidf_vectorizer_vectors, y_train)



# Transform test data
tfidf_test_vectors = fitted_vectorizer.transform(X_test)

# Predictions
predictions = model.predict(tfidf_test_vectors)

# Calculate MAE
mae = mean_absolute_error(y_test, predictions)
print(f"Mean Absolute Error (MAE): {mae}")

Mean Absolute Error (MAE): 33218.01516267072


/Users/kevinqueally/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:1549: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


In [72]:
linkedin = pd.read_csv("linkedin.csv")
linkedin.drop_duplicates(inplace=True)
linkedin['JCL'] = linkedin['jobtitle'] + ' ' + linkedin['company_name'] + ' ' + linkedin['location']
linkedin

,urn,email,company_name,distance,jobtitle,location,name,JCL
0,ACoAACAscakBTDFoTumnmGyWkkXp-udE1hfrvOg,NaN,KAYAK,DISTANCE_2,Software Engineer @ KAYAK,"Boston, MA",Maanasa Narayan,"Software Engineer @ KAYAK KAYAK Boston, MA"
1,ACoAABpbo10BvbsIRZU0j-6vGXsVxig-pymbw3o,NaN,Akoya,DISTANCE_2,Software Engineer at Akoya| MS in Information ...,"Boston, MA",Surbhi Parihar,Software Engineer at Akoya| MS in Information ...
2,ACoAACdzossBLkU64RVHXyfKabLqCg2s4r8s41k,NaN,Oracle,DISTANCE_2,Software Engineer | Oracle Health Sciences,"Boston, MA",Apoorva K,Software Engineer | Oracle Health Sciences Ora...
3,ACoAAC46yGUB_FG3ZdDpzym2dmLPDAWjcJuYaLg,NaN,Netflix,DISTANCE_2,Software Engineer @Netflix,Greater Boston,Cherie Wang,Software Engineer @Netflix Netflix Greater Boston
4,ACoAACr4v64BumYn0D750h8l6y5bWqoWAE1GRu4,NaN,Scale AI,DISTANCE_2,Software Engineer at Scale AI,Greater Boston,Kevin Li,Software Engineer at Scale AI Scale AI Greater...
...,...,...,...,...,...,...,...,...
95,ACoAAC11loMBoZIzrHaJjVAmN-Ho8yWaQxajz1w,NaN,Bacchanal Special Events,DISTANCE_2,Software Engineer,"New York, NY",Kofi Meighan,Software Engineer Bacchanal Special Events New...
96,ACoAACaDbx4Bsa0HsgkITfvaeMTx5Uf_NiFJsyo,NaN,DacheQL,DISTANCE_2,Software Engineer,New York City Metropolitan Area,Sandy Liu,Software Engineer DacheQL New York City Metrop...
97,ACoAACSyax8Bow3JBe47t5WQu0Tw01Jd9QyldJM,NaN,Bloomberg,DISTANCE_2,Software Engineer,United States,Brian Peinado,Software Engineer Bloomberg United States
98,ACoAAA7gNxYB5LsDxVvhoSqGqHntN4gWX7WHzRE,NaN,Setra Systems,DISTANCE_2,Software Engineer | CS @ UPenn,Greater Boston,Mai Nguyen,Software Engineer | CS @ UPenn Setra Systems G...


In [84]:
# Vectorize 'JLC' column in the 'linkedin' dataframe
linkedin.dropna(subset=['JCL'], inplace=True)
linkedin_text = linkedin['JCL']
linkedin_tfidf_vectors = fitted_vectorizer.transform(linkedin_text)

# Use the trained model to predict salaries for 'linkedin'
predicted_salaries = model.predict(linkedin_tfidf_vectors)

# Add the predicted salaries to the 'linkedin' dataframe as a new column 'PREDICTED_SALARY'
linkedin['PREDICTED_SALARY'] = predicted_salaries
linkedin = linkedin.sort_values(by='PREDICTED_SALARY', ascending=False)

linkedin

,urn,email,company_name,distance,jobtitle,location,name,JCL,PREDICTED_SALARY
35,ACoAAABIBy4BGqQJNd-qjnKhyXwYbjFuHBcPoLU,NaN,Google,DISTANCE_2,Senior Staff Software Engineer at Google,"Lexington, MA",Ben Vandiver,Senior Staff Software Engineer at Google Googl...,168468.584475
42,ACoAADA8vnkB0rdDdJp1AfV3g1ivEW3AYk8aGq4,NaN,Discord,DISTANCE_2,Software Engineer at Discord,"New York, NY",Peter Cho,"Software Engineer at Discord Discord New York, NY",163557.276919
95,ACoAAC11loMBoZIzrHaJjVAmN-Ho8yWaQxajz1w,NaN,Bacchanal Special Events,DISTANCE_2,Software Engineer,"New York, NY",Kofi Meighan,Software Engineer Bacchanal Special Events New...,163557.276919
93,ACoAACF2tcoB4xItAS9P35FFZUU9b_A_mUOIew0,NaN,Meta,DISTANCE_2,Software Engineer @ Meta | Harvard,"New York, NY",Lily K.,Software Engineer @ Meta | Harvard Meta New Yo...,163557.276919
17,ACoAACwfOLwBsgwmJOvXmCRxcD-q0e_AAmXBZ6M,NaN,Google,DISTANCE_2,Software Engineer at Google,"New York, NY",Nikhil Gorrepati,"Software Engineer at Google Google New York, NY",163557.276919
...,...,...,...,...,...,...,...,...,...
28,ACoAABzBv7IBvrnl9cYdVU4Gl1_yG-cHWKED7CA,NaN,Analog Devices,DISTANCE_2,"Engineer, Software Tools Development at Analog...","Medford, MA",Ashish Gawali,"Engineer, Software Tools Development at Analog...",117224.035229
50,ACoAACf-JYYBUjMotqDu5ABZbz63VLLlDfxAFck,NaN,Amazon Fulfillment Technologies & Robotics,DISTANCE_2,Actively looking for full time Software Engine...,"Boston, MA",Archie Sanghavi,Actively looking for full time Software Engine...,115678.217199
12,ACoAACI4qZsBfrB4EL3e1I9p3onzjvug2Y6yy_s,NaN,"Nextera Robotics (MIT, Y-Combinator)",DISTANCE_2,Robotics Software Engineer @ Nextera Robotics ...,Greater Boston,Rishabh Mukund,Robotics Software Engineer @ Nextera Robotics ...,112655.911228
38,ACoAACfNoQkBgKR7u37cvc9bhCw_nk637ry7tJs,NaN,Amazon Web Services (AWS),DISTANCE_2,Software Development Engineer @ Amazon Web Ser...,"Boston, MA",Unnati Kolhe,Software Development Engineer @ Amazon Web Ser...,102013.596850


In [119]:
job = [" San Fransisco CA"]
jobtitle_vector = fitted_vectorizer.transform(job)

# Use the trained model to predict salaries for 'linkedin'
predicted_job_salary = model.predict(jobtitle_vector)
predicted_job_salary

array([116376.45274407])

## Exploratory Data Analysis

In [43]:
# Extracting state codes using regex
state_codes = jobs_filtered['LOCATION'].str.extract(r'([A-Z]{2})')

# Counting occurrences of state codes
state_code_counts = state_codes[0].value_counts()

print(state_code_counts)

mean_salary = round(jobs_filtered['YEARLY_SALARY'].mean(), 2)
median_salary = jobs_filtered['YEARLY_SALARY'].median()
stdv_salary = round(jobs_filtered['YEARLY_SALARY'].std(), 2)
min_salary = jobs_filtered['YEARLY_SALARY'].min()
max_salary = jobs_filtered['YEARLY_SALARY'].max()

print("Average salary nationwide in the US: $59428")
print(f"Mean Dataset Salary: ${mean_salary}")
print(f"Median Dataset Salary: ${median_salary}")
print(f"Standard Deviation Dataset of Salary: ${stdv_salary}")
print(f"Min Dataset Salary: ${min_salary}")
print(f"Max Dataset Salary: ${max_salary}")

CA    5721
NY    3339
CO    2821
WA    2350
MA    1495
TX     962
IL     843
GA     228
FL      97
PA      76
RI      48
AZ      39
NV      39
IN      12
AF       3
NH       2
NJ       1
Name: 0, dtype: int64
Average salary nationwide in the US: $59428
Mean Dataset Salary: $104861.86
Median Dataset Salary: $88920.0
Standard Deviation Dataset of Salary: $61746.78
Min Dataset Salary: $9880.0
Max Dataset Salary: $665600.0


In [44]:
bin_counts = jobs_filtered['SALARY_BIN'].value_counts()
bin_counts

$60,000 - $80,000      3542
$40,000 - $60,000      3294
$80,000 - $100,000     2950
$100,000 - $120,000    2244
$140,000 - $180,000    2004
Over $180,000          1805
$120,000 - $140,000    1669
Under $40,000           717
Name: SALARY_BIN, dtype: int64